### Q1. Continuing the Vector

Our Vector implementation so far looks like this

In [13]:
from doctest import run_docstring_examples as dtest

First, implement all the ad-hoc tests so far as doctests. Then implement multiplication and subtraction for this class. Notice that this is currently a non-mutable vector.

In [82]:
#example from Fluent
from array import array 
import reprlib
import math
import numbers
import functools
import operator
import itertools

class Vector:
    """
    Parameters
    ----------
    Returns
    -------
    Notes
    -----

    Examples
    --------
    >>> v3=Vector([1,2,3])
    >>> v2=Vector([3,4])
    >>> v1=Vector([1,2])
    >>> v4=Vector([3,4])
    >>> v1+1,v1+[1,2]
    (Vector([2.0, 3.0]), Vector([2.0, 4.0]))
    >>> v1 + v2, v2 + v1
    (Vector([4.0, 6.0]), Vector([4.0, 6.0]))
    >>> v2==v4
    True
    >>> v1==v2
    False
    >>> list(v1)
    [1.0, 2.0]
    >>> -v1
    Vector([-1.0, -2.0])
    >>> +v1
    Vector([1.0, 2.0])
    >>> v1 + [1,2]
    Vector([2.0, 4.0])
    >>> [1,2] + v1
    Vector([2.0, 4.0])
    >>> v1 + 3
    Vector([4.0, 5.0])
    >>> 3 + v1
    Vector([4.0, 5.0])
    >>> v1 += 3
    >>> v1
    Vector([4.0, 5.0])
    >>> list(iter(v1))
    [4.0, 5.0]
    >>> len(v1)
    2
    >>> v1[1]
    5.0
    >>> abs(v1)
    6.4031242374328485
    >>> v1 * v2
    Vector([12.0, 20.0])
    >>> v2* v1
    Vector([12.0, 20.0])
    >>> 4*v1
    Vector([16.0, 20.0])
    >>> v1*4.9
    Vector([19.6, 24.5])
    """
    
    typecode = 'd'
    
    def __init__(self, components):
        self._components = array(self.typecode, components)
        
    def __iter__(self):
        return iter(self._components)
    
    def __repr__(self):
        components = reprlib.repr(self._components) 
        components = components[components.find('['):-1] 
        return 'Vector({})'.format(components)
    
    def __eq__(self, other):
        if isinstance(other, Vector):
            return (len(self) == len(other) and
                all(a == b for a, b in zip(self, other)))
        else:
            return NotImplemented
    
    def __abs__(self):
        return math.sqrt(sum(x * x for x in self))
    
    def __bool__(self): 
        return bool(abs(self))
    
    def __len__(self):
        return len(self._components)
    
    def __getitem__(self, index): 
        cls = type(self)
        
        if isinstance(index, slice):
            return cls(self._components[index])
        elif isinstance(index, numbers.Integral): 
            return self._components[index]
        else:
            msg = '{.__name__} indices must be integers' 
            raise TypeError(msg.format(cls))
            
    def __neg__(self):
        return Vector(-x for x in self) 
    
    def __pos__(self):
        return Vector(self)
    
    def _check_length_helper(self , rhs):
        if not len(self)==len(rhs):
            raise ValueError(str(self)+' and '+str(rhs)+' must have the same length')
    
    def __add__(self, rhs):
        try:
            if isinstance(rhs, numbers.Real):
                return Vector(a + rhs for a in self) 
            else: #
                self._check_length_helper(rhs)
                pairs = zip(self, rhs)
                return Vector(a + b for a, b in pairs)
        except TypeError:
            raise NotImplemented
    
    def __radd__(self, other): # other + self delegates to __add__
        return self + other

    def __mul__(self, rhs):
        try:
            if isinstance(rhs, numbers.Real):
                return Vector(a * rhs for a in self) 
            else: #
                self._check_length_helper(rhs)
                pairs = zip(self, rhs)
                return Vector(a*b for a, b in pairs)
        except TypeError:
            raise NotImplemented
    
    def __rmul__(self, other): # other + self delegates to __add__
        return self * other
    
    def __sub__(self, rhs):
        try:
            if isinstance(rhs, numbers.Real):
                return Vector(a-rhs for a in self) 
            else: #
                self._check_length_helper(rhs)
                pairs = zip(self, rhs)
                return Vector(a-b for a, b in pairs)
        except TypeError:
            raise NotImplemented
    
    def __rsub__(self, other): # other + self delegates to __add__
        return self - other

In [84]:
dtest(Vector, globals(), verbose=True)

Finding tests in NoName
Trying:
    v3=Vector([1,2,3])
Expecting nothing
ok
Trying:
    v2=Vector([3,4])
Expecting nothing
ok
Trying:
    v1=Vector([1,2])
Expecting nothing
ok
Trying:
    v4=Vector([3,4])
Expecting nothing
ok
Trying:
    v1+1,v1+[1,2]
Expecting:
    (Vector([2.0, 3.0]), Vector([2.0, 4.0]))
ok
Trying:
    v1 + v2, v2 + v1
Expecting:
    (Vector([4.0, 6.0]), Vector([4.0, 6.0]))
ok
Trying:
    v2==v4
Expecting:
    True
ok
Trying:
    v1==v2
Expecting:
    False
ok
Trying:
    list(v1)
Expecting:
    [1.0, 2.0]
ok
Trying:
    -v1
Expecting:
    Vector([-1.0, -2.0])
ok
Trying:
    +v1
Expecting:
    Vector([1.0, 2.0])
ok
Trying:
    v1 + [1,2]
Expecting:
    Vector([2.0, 4.0])
ok
Trying:
    [1,2] + v1
Expecting:
    Vector([2.0, 4.0])
ok
Trying:
    v1 + 3
Expecting:
    Vector([4.0, 5.0])
ok
Trying:
    3 + v1
Expecting:
    Vector([4.0, 5.0])
ok
Trying:
    v1 += 3
Expecting nothing
ok
Trying:
    v1
Expecting:
    Vector([4.0, 5.0])
ok
Trying:
    list(iter(v1))
Expect

### Q2. Mixins for functionality

Here is a set of methods that logs dictionary access

In [86]:
class LoggedMappingMixin: 
    '''
    Add logging to get/set/delete operations for debugging. 
    '''
    __slots__ = ()
    def __getitem__(self, key): 
        print('Getting ' + str(key)) 
        return super().__getitem__(key)
    def __setitem__(self, key, value): 
        print('Setting {} = {!r}'.format(key, value)) 
        return super().__setitem__(key, value)
    def __delitem__(self, key): 
        print('Deleting ' + str(key)) 
        return super().__delitem__(key)

Notice the use of `super()` here. `super()` is the same as `super(self.__class__, self)`. But we dont have a parent!

What is going on? You tell me the answer to this when you inherit a `LoggedDict` with no implementation from both `LoggedMappingMixin` and `dict`. Which order must you inherit in? Play with the `mro` method and figure this out.

In [127]:
#write the LoggedDict class 2 ways and play with the mro, then write the reason

class LoggedDict(LoggedMappingMixin, dict): 
    pass

In [128]:
LoggedDict.mro()

[__main__.LoggedDict, __main__.LoggedMappingMixin, dict, object]

In [126]:
ba = LoggedDict({'a':3, 'b':4, 'c':5})
print(ba['a'])
ba['d']=10
ba

Getting a
3
Setting d = 10
Getting a
Getting b
Getting c
Getting d


{'a': 3, 'b': 4, 'c': 5, 'd': 10}

In [103]:
#your code here
class LoggedDict(dict, LoggedMappingMixin): 
    pass

In [97]:
LoggedDict.mro()

[__main__.LoggedDict, dict, __main__.LoggedMappingMixin, object]

In [104]:
ba = LoggedDict({'a':3, 'b':4, 'c':5})
print(ba['a'])
ba['d']=10
ba

*your answer here*
We need to inherit from dict after LoggedMappingMixin, because lookup will search the mro left to right, and we want to find LoggedMappingMixin before we find dict so that dict's behavior is overridden. 

### Q3. The Pavlos Problem

ABC's and doctests. The Pavlos problem.

Introspection of a class hierarchy is helped by:
`__subclasses__()` and `_abc_registry` which give us concrete subclasses and virtual subclasses respectively. We can use this to fully document an interface via an example.

In [179]:
import abc
class StackInterface(abc.ABC):
    """
    >>> a = ListStack()
    >>> a.push(1)
    >>> a.push(2)
    >>> a.peek()
    2
    >>> a.pop()
    2
    >>> a.pop()
    1
    >>> a.peek()
    >>> a.pop()
    """
    
    @abc.abstractmethod
    def push(self, value):
        "Push value onto the stack. Return None"
        
    @abc.abstractmethod
    def pop(self):
        "Pop value from Stack. Return None if nothingon stack"
        
    @abc.abstractmethod
    def peek(self):
        "Peeak at top of stack. Return None if empty"

Implement `ListStack` using a python list

In [185]:
#your code here
class ListStack(StackInterface):
    def __init__(self):
        self._stack = list()
        
    def push(self, value):
        self._stack.append(value)
        
    def pop(self):
        try: 
            popped = self._stack[-1]
            self._stack = self._stack[:-1]
            return popped
        except: 
            "Empty ListStack"
    
    def peek(self):
        try:
            return self._stack[-1]
        except: "Empty ListStack"
    
    def __getitem__(self, idx):
        return self._stack[idx]
    
    def __iter__(self):
        return iter(self._stack)
    
    def __repr__(self):
        return "ListStack(" + str(self._stack) + ")"

In [187]:
a = ListStack()
a.push(1)
print(a)
print(a.peek())
print(a.pop())
print(a)

ListStack([1])
1
1
ListStack([])


How do we test this using the tests in `StackInterface`? And in general for other virtual or real subclasses? Show this here. (work out doing this from a file at home, you dont need to answer the file case here). This recipe wont work with py.test

In [186]:
globaldict = globals()
globaldict['Stack']=StackInterface.__subclasses__()
dtest(StackInterface, globaldict, verbose=True)

Finding tests in NoName
Trying:
    a = ListStack()
Expecting nothing
ok
Trying:
    a.push(1)
Expecting nothing
ok
Trying:
    a.push(2)
Expecting nothing
ok
Trying:
    a.peek()
Expecting:
    2
ok
Trying:
    a.pop()
Expecting:
    2
ok
Trying:
    a.pop()
Expecting:
    1
ok
Trying:
    a.peek()
Expecting nothing
ok
Trying:
    a.pop()
Expecting nothing
ok


### Q4. Your Timeseries Project

Operator overloading on your `TimeSeries` class.

Your `TimeSeries` class should be, by now, a well documented, well tested, mutable, class which implements:

- `__getitem__`: to get a value for a given time
- `__setitem__`: set the value for the given time
- `__contains__`: is a value in the values
- `__iter__`: iterates over values. (This might have iterated over tuples of (time, value) pairs earlier
- `values`: returns a numpy array of values
- `itervalues`: returns an iterator over them
- `times`: returns a numpy array of times
- `itertimes`: returns an iterator over them
- `items`: returns a list of time-value tuple pairs
- `iteritems`: returns an iterator over these
- `__len__`: returns a length.
- `__repr__`: abbreviating spring representation

Add to these methods(again well tested):

- infix addition, subtraction, equality and multiplication. Here you must check that the lengths are equal and that the time domains are the same for the case of the operations on a TimeSeries (the latter implies the former). Return a `ValueError` in case this fails:

`ValueError(str(self)+' and '+str(rhs)+' must have the same time points')`

Let these be elementwise operations, as we might expect from a numpy array-like thing. As before, handle the case of a constant.
- unary `__abs__`, `__bool__`, `__neg__`, and `__pos__` with the same semantics as the `Vector` class above.


A question that might arise is what to do if we add numpy arrays or regular python lists. These should fail with `raise NotImplemented` as we dont have time associated. An option might have been to associate the array with the time indexing of the other array, but this is making too many assumptions: the user can do this explicitly.

You will probably have to catch another exception for this to happen.

Put this code into your project repo.

### TO READ: Numpy ufuncs and function overloading

Check this out. Read http://docs.scipy.org/doc/numpy-dev/reference/arrays.classes.html#special-attributes-and-methods to understand how this works. We will use it later.

In [188]:
import numpy as np
import pandas as pd
p=pd.Series([1,2,3])
print(type(p))
p2=np.exp(p)
p2, type(p2)

<class 'pandas.core.series.Series'>


(0     2.718282
 1     7.389056
 2    20.085537
 dtype: float64, pandas.core.series.Series)